This file takes the original dataset from Kaggle and converts the dataset into Python-readable values

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import math
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from tabulate import tabulate
import sys
import copy
sys.path.append('C:\\Users\\chery\\Documents\\SeattlePolice911Response')
from utils.baseML import BaseML
from datetime import datetime
from sklearn import preprocessing

Read in CSV

In [ ]:
spir_original = pd.read_csv("Seattle_Police_Department_911_Incident_Response.csv")

In [ ]:
print(spir_original.columns)

 Drop unused Columns

In [ ]:
drop_columns= ['CAD CDW ID', 'CAD Event Number', 'General Offense Number', 'Hundred Block Location', 'District/Sector',
       'Zone/Beat', 'Longitude', 'Latitude',
       'Incident Location']

spir_drop_unused_col = spir_original.drop(labels=drop_columns, axis=1)

Create list of unique census tracts. This is used to pull data from the US Census API 5 year American Community Survey

In [ ]:
census_tracts = spir_drop_unused_col["Census Tract"].unique()
census_tracts = pd.DataFrame(census_tracts, columns=["census_tracts"])
census_tracts.dropna(axis =0, inplace=True)
census_tracts.census_tracts=census_tracts.census_tracts.apply(format_tract)
census_tracts.census_tracts = pd.Series(census_tracts.census_tracts.unique())
census_tracts.dropna(axis=0, inplace=True)
census_tracts.to_csv("census_tracts.csv")

Drop rows with missing values

In [ ]:
spir_dropna = spir_drop_unused_col.dropna(axis=0, how='any')

Fix date and time

In [ ]:
class Seattle_Police_ML(BaseML):

    def dayDifference(self, fdate, ldate):
        delta = ldate - fdate
        return(delta)

    #Transform At Scene Time and Event Clearance Date to datetime objects. Create time_at_scene parameter
    def transform_times(self):
        dt_format = '%m/%d/%Y %I:%M:%S %p'
        self.df['at_scene_time'] = self.df["At Scene Time"].apply(lambda x: datetime.strptime(x, dt_format))
        self.df['event_clear_time']=self.df["Event Clearance Date"].apply(lambda x: datetime.strptime(x, dt_format))
        self.df['time_at_scene'] = self.df.apply(lambda x: pd.to_timedelta(self.dayDifference(x["at_scene_time"], x["event_clear_time"])), axis = 1)
        self.df.drop(["At Scene Time", "Event Clearance Date"], axis = 1, inplace = True)
        print(self.df.sample(3))

    #Bins the time_at_scene data. Unused for this project
    def simplify_times(self):
        bins = (
            pd.Timedelta(minutes = 0),
            pd.Timedelta(minutes = 30),
            pd.Timedelta(minutes = 60),
            pd.Timedelta(hours = 4),
            pd.Timedelta(hours = 8),
            pd.Timedelta(hours = 12),
            pd.Timedelta(days = 1),
            pd.Timedelta(days = 36500)
        )

        labels = ['< 30min', '30-60min','1-4hrs', '4-8hrs','8-12hrs','12-24hrs', '24hrs+']

        return pd.cut(self.df['time_at_scene'], bins, labels = labels)

    def map_func(self,parameter_df, x):
        try:
            return parameter_df.loc[math.floor(x["Census Tract"]), str(datetime.strptime(x.at_scene_time, "%Y-%m-%d %H:%M:%S").year)]
        except KeyError:
            return None
            pass
            #print(math.floor(x["Census Tract"]))

    #Uses CSV files with census data to fill in parameters in the dataset
    def add_census_data(self, census_filename, parameter_name):
        parameter_df = pd.read_csv(census_filename, index_col=1)
        print(self.df["Census Tract"].sample(3))
        self.df[parameter_name]=self.df.apply(lambda x: self.map_func(parameter_df, x), axis=1)
        
    #normalizes features    
    def normalize_features(self):
        scaler = preprocessing.StandardScaler().fit(self.df)
        df_scaled = scaler.transform(self.df)
        self.df_normalized = pd.DataFrame(df_scaled, columns = self.df.columns, dtype= 'int64')


In [ ]:
spir_final = Seattle_Police_ML()
spir_final.df = copy.deepcopy(spir_dropna)
spir_final.transform_times()
spir_final.simplify_times()


In [ ]:
spir_final.df.to_csv("SPIR_transformed_times.csv")

In [ ]:
spir_final = Seattle_Police_ML("SPIR_transformed_times.csv")
print(spir_final.df["Census Tract"].sample(3))
spir_final.add_census_data("white_percent.csv","white")


In [ ]:
spir_final.add_census_data("total_pop_by_tract.csv","total_pop")
spir_final.add_census_data("male_percent.csv","male")
spir_final.add_census_data("notUScitizen_percent.csv","notUScitizen")
spir_final.add_census_data("asian_percent.csv","asian")
spir_final.add_census_data("black_percent.csv","black")
spir_final.add_census_data("native_percent.csv","native")
spir_final.add_census_data("other_race_percent.csv","other_race")
spir_final.add_census_data("two_races_percent.csv","two_races")
spir_final.add_census_data("some_college_percent.csv","some_college")
spir_final.add_census_data("bachelors_percent.csv","bachelors")
spir_final.add_census_data("grad_deg_percent.csv","grad_deg")
spir_final.add_census_data("under18MC_percent.csv","under18MC")
spir_final.add_census_data("under18MS_percent.csv","under18MS")
spir_final.add_census_data("under18FS_percent.csv","under18FS")
spir_final.add_census_data("under18T_percent.csv","under18T")
spir_final.add_census_data("income_by_tract.csv", "income")


In [ ]:
spir_final.df.to_csv("SPIR_census_added.csv")


In [ ]:
spir_final.df.dropna(axis=0, inplace=True)

In [ ]:
spir_final.df.drop(["Unnamed: 0"], axis=1, inplace=True)

<h4>Encode "Initial Type Description" parameter</h4>

In [ ]:
spir_final.df['Initial Type Description'] = spir_final.df['Initial Type Description'].astype('category')
print(spir_final.df.dtypes)

In [ ]:
spir_final.df["initial_type_desc_cat"] = spir_final.df["Initial Type Description"].cat.codes
print(spir_final.df.head())

<h4>Create additional parameters for time of day (AM or PM), day of the week, month, and year. Convert time_at_scene to seconds</h4>

In [ ]:
spir_final.df.at_scene_time = spir_final.df.at_scene_time.astype('datetime64')
spir_final.df.event_clear_time = spir_final.df.event_clear_time.astype('datetime64')
spir_final.df.time_at_scene = pd.to_timedelta(spir_final.df.time_at_scene)

print(spir_final.df.time_at_scene.dtypes)

In [ ]:
noon = datetime(2021, 11, 12,hour = 12)
spir_final.df["at_scene_time_pm"] = spir_final.df.at_scene_time.apply(lambda x: 0 if x.hour < noon.hour else 1)
spir_final.df["event_clear_time_pm"] = spir_final.df.event_clear_time.apply(lambda x: 0 if x.hour < noon.hour else 1)


In [ ]:
spir_final.df["at_scene_time_weekday"] = spir_final.df.at_scene_time.apply(datetime.weekday)
spir_final.df["event_clear_time_weekday"] = spir_final.df.event_clear_time.apply(datetime.weekday)

In [ ]:
spir_final.df["at_scene_time_month"] = spir_final.df.at_scene_time.apply(lambda x: x.month)
spir_final.df["event_clear_time_month"] = spir_final.df.event_clear_time.apply(lambda x: x.month)

In [ ]:
spir_final.df["at_scene_time_year"] = spir_final.df.at_scene_time.apply(lambda x: x.year)
spir_final.df["event_clear_time_year"] = spir_final.df.event_clear_time.apply(lambda x: x.year)

In [ ]:
spir_final.df.sample(10)

In [ ]:
from datetime import timedelta
spir_final.df["time_at_scene_seconds"] = spir_final.df.time_at_scene.apply(timedelta.total_seconds)

In [ ]:
spir_final.df.to_csv("SPIR_encoded_2021116.csv")